# Youbike 使用量預測

## 研究背景

Youbike（Ubike）是台北市的一個共享單車系統，為市民提供便捷的短程交通解決方案。本專案的主要目標是透過資料科學方法，評估未安裝Youbike站點的潛在使用量，以提供城市規劃者更有依據的站點選擇。透過分析 Youbike 2.0 的借車/還車流量，結合站點附近的地理區位和時空資料，我們將建立一個預測模型，探討站點安裝後可能的使用趨勢。

> #### 為何選擇 Youbike 2.0? 
> 目前全台北市已全面改為 Youbike 2.0。因為停車柱樁無需串接電源以及網路， 2.0 的設點可以更為便利。以下探討之流量，皆為 Youbike 2.0 的數據。

## 專案目的

1. **評估未安裝 Youbike 站點的潛在使用量：** 透過歷史借車/還車流量數據，我們將探索哪些區域可能受益於新增Youbike站點，以提高共享單車的使用率。

2. **探討地理區位的影響：** 分析站點附近的地理區位資訊，例如是否靠近捷運站、是否有便利超商、是否在學校附近等，以瞭解這些因素對使用量的影響。

3. **整合時空資料進行預測：** 考慮時空因素，例如天氣和時間，以建立更準確的預測模型，預測未來 Youbike 使用量的變化。

## 專案方法

1. **資料收集：** 收集自 2020 年至 2023 年的 Youbike 借車/還車流量數據，包括每個站點的小時單位細節。

2. **地理區位分析：** 分析每個站點周邊地理區位，包括捷運站、便利超商、學校等，並建立相應的特徵。

3. **時空資料整合：** 整合時空資料，包括每個站點在不同時刻的天氣狀況，以及其他可能影響使用量的時間相關因素。

4. **機器學習模型訓練：** 使用機器學習算法，利用歷史資料訓練預測模型，並評估模型的效能。

# 資料概覽

以下簡單介紹資料來源以及型態。特徵工程的細節演算法，請參照`transformation/features.py/VillageFeature`

In [13]:
import pandas as pd
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')

## YB 相關 -- 流量

從政府開訪資料網站取得之臺北市公共自行車 2.0 租借紀錄，經過小時加總後得到每一個 Youbike 站點在該小時的借出與還車數量

In [14]:
yb_flow = pd.read_parquet("DATA/Youbike/youbike_flow_data_network.pq", engine='pyarrow').convert_dtypes(dtype_backend="pyarrow")[['name','time', 'count_out', 'count_in']]
yb_flow.sample(n=10)

,name,time,count_out,count_in
9829224,實踐國中,2021-09-21 00:00:00,0,0
14323423,新生長春路口,2022-06-28 13:00:00,2,4
16166367,中429公園,2020-05-23 21:00:00,0,0
23521583,捷運松山站(4號出口),2021-10-10 14:00:00,11,13
734688,臺大男七舍前,2022-08-16 03:00:00,0,0
9174765,景美醫院,2022-09-08 21:00:00,7,2
7468503,延平北路八段133巷口,2022-12-02 18:00:00,1,1
19557759,下灣公園(成功路),2020-06-05 00:00:00,0,0
2848678,仁愛醫院,2022-05-13 22:00:00,1,1
25230490,八德路四段652巷口,2021-11-04 22:00:00,0,0


## YB 相關 -- 資訊

Youbike 站點資訊包含經緯度座標、總站點數量、以及地址等資訊，可以從即時 youbike2.0 的 API 中取得。經轉換後資料如下：

In [15]:
yb_info = pd.read_parquet("DATA/Youbike/yb_info.pq").convert_dtypes(dtype_backend='pyarrow')
yb_info.sample(n=10)

,name,total,lat,lng,address
519,林森北平路口,20,25.04659,121.52377,林森北路與北平東路口西北側
536,重慶南詔安街口,15,25.02507,121.51645,重慶南路三段/詔安街
202,國順公園,20,25.06594,121.51078,昌吉街117號前
1097,臺北醫學大學,55,25.02668,121.56175,台北醫學大學(吳興街220巷59弄)
222,樹德公園_1,21,25.06669,121.51615,大龍街/大龍街85巷(東北側)
89,捷運大安森林公園站(2號出口),62,25.03332,121.53435,捷運大安森林公園站(2號出口)西側
886,承德石牌路口,17,25.11272,121.50922,承德路七段與石牌路一段口北側
514,杭州南路一段101巷口,14,25.03746,121.52799,金山南路一段72號前
28,臺大男七舍前,20,25.01537,121.54698,長興街/長興街80巷口西側
745,安泰街83巷,20,25.07669,121.61709,安泰街83巷3號(對面)


## 人口學變量 -- 各里各年齡層比例 + 收入分佈

人口結構可能會影響使用 Youbike 的情形。壯年為主的區域使用率應該可以預期較老年為主之區域有較高的使用量，但中高齡中使用 Youbike 進行運動之情形也有可能。由於不清楚實際使用情況，我們將年齡分佈區分為幾個主要區段，分別捕捉青少年、大學生、上班族、壯年、老年、高齡，藉由機器學習的方式判斷規則。

另外，除了人口結構，收入分佈或許也是影響因子之一。高所得之區域自用車上下班情況或許高過低所得區域，而高所得區域同時作為辦公區的機率也較高，因此可預期還車量將高於借車輛。這些因素尚與一地區的繁華程度交互影響，因此也將藉由機器學習的方式處理。

In [16]:
demographic = pd.read_parquet("DATA/Demographic/demographic.pq")
demographic.drop(columns=["行政區", "綜合所得總額"], inplace=True)

In [17]:
demographic

,里別,總計,age_0_15,age_16_18,age_19_24,age_25_40,age_41_65,age_66_75,age_76_100,平均數,中位數,第一分位數,第三分位數
0,莊敬里,5045.0,11.912785,2.418236,5.748266,20.118930,38.691774,13.280476,7.829534,844,543,242,1054
1,東榮里,7799.0,17.348378,2.295166,5.064752,17.027824,36.145660,13.437620,8.680600,1512,723,287,1758
2,三民里,6380.0,12.351097,2.147335,5.564263,17.789969,38.605016,13.620690,9.921630,1115,625,260,1320
3,新益里,4326.0,11.534905,2.265372,4.808137,20.850670,37.586685,14.193250,8.760980,919,538,231,1181
4,富錦里,4942.0,14.225010,2.185350,4.917038,19.607446,36.806961,14.467827,7.790368,1118,580,262,1346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,關渡里,10982.0,11.946822,2.522309,5.554544,22.272810,37.606993,12.793662,7.302859,859,482,218,1007
452,泉源里,2267.0,7.190119,2.514336,6.793119,21.614468,40.979268,13.056903,7.851787,717,395,190,724
453,湖山里,1492.0,6.300268,2.077748,4.423592,19.235925,41.085791,14.946381,11.930295,967,452,216,980
454,大屯里,1232.0,11.444805,1.948052,6.250000,20.535714,37.662338,12.662338,9.496753,887,457,237,997


### 各里地理資訊
以上為各里之人口學變量資訊，要與 Youbike 2.0 之站點連結，尚需各里之邊界座標。

若一站點位於里的正中心，則該站點被分配到該里的人口學結構。然而若位於各里的邊界，甚至交界處，則判斷上會叫複雜。為簡化判斷，我們採取基於各里重心的逆距離加權法(inverse distance weighting, IDW)
1. 里 k 的人口學變數 U 為 $U^k$
1. 計算各里之幾何重心(centroid)
1. 對於Youbike 站點 i，取得與各里 k 之間的距離 $d(i,k)$
1. 對於站點 i 的人口學變數 $U_i$，其加權值則為
$$
U_i = \frac{
\sum_{k} w_{i,k} U^k
}{
    \sum_{k} w_{i,k}
}
$$

其中 $w_{i,k} = \frac{1}{d(i,k)}$

運算細節請參考 `transformation/features.py/VillageFeature`

In [18]:
li_coord = pd.read_parquet("DATA/Demographic/li_coords_xy.pq")
li_coord

,VILLNAME,x,y
0,西新里,309680.933610,2.771898e+06
1,重陽里,310291.290886,2.772318e+06
2,蘆洲里,310928.725286,2.772775e+06
3,石潭里,309759.045690,2.772820e+06
4,朱園里,303960.451833,2.771279e+06
...,...,...,...
451,泉源里,302505.110366,2.782842e+06
452,平等里,308390.308238,2.781948e+06
453,大屯里,300992.353133,2.783640e+06
454,菁山里,307313.564606,2.783887e+06


## 捷運

捷運對 Youbike 使用量的影響很大。毫無疑問，在流量高的出口， Youbike 被借走或歸還的次數勢必也高。另外，因為從捷運轉 Youbike 想有前30分鐘免費的優惠，所以民眾會有誘因以 Youbike 進行短距離的銜接。

最能減少誤差的模型設定方法，應該是直接判斷該 Youbike 站點附近捷運站的流量。然而本專案的目的為評估未來新設站點的效益，若過度依賴捷運站即時的流量，未來進行預測時尚須主動增加此變數之值，增加評估時的複雜度。

解決方法為以附近捷運站（如果有）在該時的對數全年進站量與出站量作為加權。比方說，若該站點 d 距離內為台北車站，則該車站在晚間六點的權重則為 6.57。

In [19]:
MRT = pd.read_parquet("DATA/MRT/MRT_coord_with_flow_weight.pq")
MRT

,station,hour,in,out,No,lng,lat
0,台北車站,18,6.575462,6.605948,M5,121.516246,25.046755
1,台北車站,18,6.575462,6.605948,M6,121.516787,25.046234
2,台北車站,18,6.575462,6.605948,M7,121.518643,25.046077
3,台北車站,18,6.575462,6.605948,M8,121.517479,25.045948
4,台北車站,18,6.575462,6.605948,M1,121.518193,25.048232
...,...,...,...,...,...,...,...
9067,十四張,1,1.623249,1.748188,0,121.527701,24.984467
9068,十四張,2,1.204120,1.612784,0,121.527701,24.984467
9069,十四張,5,1.000000,0.903090,0,121.527701,24.984467
9070,十四張,4,1.000000,0.845098,0,121.527701,24.984467


因為與實際時間無關，僅與時段有關，未來於評估新站點效益時，可僅指定地點與一整天的時段，不必指定實際捷運站進出口流量。

此變數可作為繁華程度的指標之一。

## 公車

捷運站並無法全面捕捉台北市個地理位置的繁華程度，因此我們將網絡範圍更廣的公車納入資料中。

然而公車並沒有上下車流量的資料可用，因此我們使用另一種方法來捕捉公車相關的繁華程度。一般來說，交通要道上的公車站不見得會比蛋白區多，但單一公車站點所涵蓋的公車數量則會隨該地點的熱鬧程度增加。


In [20]:
bus = pd.read_parquet("DATA/Bs/bus_agg_by_stop.pq")
bus

,PublicId,Name,Latitude,Longitude,Bus_number,x,y
0,1000100040,捷運中正紀念堂站(中山),25.036430,121.516730,1,302143.790748,2.769912e+06
1,1000500040,重慶南路一段,25.045607,121.513187,2,301782.392692,2.770927e+06
2,1000500041,重慶南路一段,25.044691,121.513165,2,301780.557479,2.770826e+06
3,1000500042,重慶南路一段,25.044432,121.513114,2,301775.530123,2.770797e+06
4,1000900060,衡陽路,25.042270,121.510210,5,301483.401044,2.770557e+06
...,...,...,...,...,...,...,...
3694,2517206110,崁頂三路,25.195630,121.430072,1,293342.737269,2.787515e+06
3695,2517206260,輕軌淡水行政中心站,25.189169,121.443384,1,294686.697500,2.786804e+06
3696,2517300400,馬偕醫院,25.141655,121.459785,4,296357.683439,2.781547e+06
3697,2517300500,捷運竹圍站,25.136591,121.460120,4,296393.333479,2.780986e+06


我們將大台北地區所有公車路線以及其經過的佔，藉由爬蟲擷取下來，並統計單一站點所經過之公車數量。站點可能被區分為數個子站點，但並不影響加總。我們將各 Youbike 站點附近距離 $d$ 範圍內，所有公車子站點的公車數量加總，作為公車的指標。

## 高中

相比於大多數大學附近會設有捷運站的情況，高中則常常會與捷運站有一段距離，對於通勤上學的高中生來說，搭乘捷運至最近的捷運站後轉騎乘 Youbike 是一個較有效率的通勤方式。
對於每一個 Youbike 站點，統計附近 $d$ 距離之內，是否設有高中。

In [21]:
highschool = pd.read_parquet("DATA/Education/highschool_coord.pq")
highschool

,school_name,lat,lng
0,臺北市立西松高級中學,25.054988,121.565841
1,臺北市立中崙高級中學,25.048714,121.561104
2,臺北市立松山高級中學,25.043634,121.565614
3,臺北市立永春高級中學,25.032446,121.578128
4,國立臺灣師範大學附屬高級中學,25.033674,121.540411
...,...,...,...
134,新北市私立清傳高級商業職業學校,25.052015,121.476840
135,新北市私立能仁高級家事商業職業學校,24.957891,121.540058
136,新北市私立豫章高級工商職業學校,24.997370,121.458729
137,新北市私立莊敬高級工業家事職業學校,24.985717,121.532169


## 景點

觀光景點附近使用 Youbike 機率較高。對於每一個 Youbike 站點，統計附近 $d$ 距離之內，有多少官方定義之觀光景點。

In [22]:
tourist_spot = pd.read_parquet("DATA/Tourist/taipei_tour.pq")
tourist_spot.sample(n=10)

,tour_name,lat,lng,Class1,Class2,Class3
3767,芝山巖惠濟宮,25.10310,121.53080,4,NaN,NaN
3995,西昌街觀光夜市,25.03838,121.50060,6,NaN,NaN
4094,中央藝文公園_華山大草原遊戲場,25.04612,121.52683,18,NaN,NaN
3842,錫口(彩虹)碼頭,25.05151,121.57648,12,NaN,NaN
3975,博愛路_漢口街_相機街,25.04533,121.51139,18,NaN,NaN
3976,文昌街_家具街,25.03220,121.55647,18,NaN,NaN
4139,CITYLINK南港店,25.05264,121.60598,18,NaN,NaN
3852,華中河濱公園,25.01540,121.49476,12,13.0,15.0
3868,擎天崗,25.16727,121.57419,7,8.0,16.0
4033,新北投善光寺,25.13551,121.51890,4,NaN,NaN


### 河濱自行車道

河濱自行車道為另一種特別之觀光景點。自行車道為路徑，因此只要 Youbike 站點位於河濱自行車道一定距離之內，皆可視為單車娛樂之涵蓋範圍。我們以自行車道向外延伸 1km 的範圍內覆蓋的站點標記為「河濱車柱」，加入特徵中。


In [23]:
bike_route = pd.read_parquet("DATA/Tourist/tpe_river_bike.pq")
bike_route

,lat,lng
0,25.109929,121.467283
1,25.109890,121.467547
2,25.109818,121.467912
3,25.109660,121.468860
4,25.109676,121.468981
...,...,...
4366,25.098003,121.511199
4367,25.097981,121.511194
4368,25.097958,121.511195
4369,25.097936,121.511199


## 7-11 座標

為了更全面的捕捉繁華程度，我們將遍布大台北地區的 7-11 納入資料中。
我們將各 Youbike 站點附近距離 $d$ 範圍內，所有 7-11 的數量加總，作為便利商店的指標。

In [24]:
conv_store = pd.read_parquet("DATA/711/all_711_coord.pq")
conv_store

,store_name,lat,lng
0,上弘,25.056391,121.548287
1,小巨蛋,25.050944,121.549433
2,中崙,25.048396,121.552737
3,北體,25.050888,121.552850
4,台場,25.048086,121.551158
...,...,...,...
898,懷得,25.114096,121.519656
899,關渡,25.121540,121.467483
900,關渡站,25.125037,121.467181
901,鐏賢,25.117453,121.506854


## 天氣

考量到當天氣太冷、太熱或是下雨天，皆會影響到騎乘 Youbike 的意願，因此我們將 2020/4 ~ 2023/12 大台北地區歷史天氣資料藉由爬蟲擷取下來，其中氣象觀測站包含「信義」、「臺北」、「竹子湖」、「社子」、「石牌」、「天母」、「平等」、「內湖」、「松山」、「文山」。
資料包含了氣溫、濕度、風速、雨量、以及各觀測站的經緯度。

In [25]:
weather_df = pd.read_csv("DATA/Weather/weather_data_with_coord.csv", parse_dates=['datetime']).convert_dtypes(dtype_backend='pyarrow')

In [26]:
weather_df.sort_values(['station_name', 'datetime'])

,datetime,temperature,relative_humidity,wind_speed,precipitation,station_id,station_name,station_type,latitude,longitude
289200,2020-04-01 01:00:00,20.0,97,1.1,1.0,C0AC70,信義,auto_C0,25.0378,121.5646
289202,2020-04-01 02:00:00,19.3,96,1.1,0.0,C0AC70,信義,auto_C0,25.0378,121.5646
289204,2020-04-01 03:00:00,18.7,96,0.3,0.5,C0AC70,信義,auto_C0,25.0378,121.5646
289206,2020-04-01 04:00:00,18.3,96,0.9,0.5,C0AC70,信義,auto_C0,25.0378,121.5646
289208,2020-04-01 05:00:00,18.3,94,1.0,0.0,C0AC70,信義,auto_C0,25.0378,121.5646
...,...,...,...,...,...,...,...,...,...,...
52555,2023-12-31 20:00:00,18.4,67,4.2,0.0,466920,臺北,cwb,25.0377,121.5149
52556,2023-12-31 21:00:00,18.2,64,4.1,0.0,466920,臺北,cwb,25.0377,121.5149
52557,2023-12-31 22:00:00,17.9,65,4.2,0.0,466920,臺北,cwb,25.0377,121.5149
52558,2023-12-31 23:00:00,17.6,61,4.7,0.0,466920,臺北,cwb,25.0377,121.5149


In [27]:
obs = weather_df.groupby('station_name').first().reset_index()
px.scatter_mapbox(obs, lat = 'latitude', lon = 'longitude', hover_name='station_name', mapbox_style='carto-positron')

# Adding Features

In [28]:
from transformation import FeaturePipe
import transformation.features as FE

yb_flow_set_index = yb_flow.set_index(['name', 'time'])
yb_info_set_index = yb_info.set_index('name')


In [29]:

fp = FeaturePipe([
    FE.XYCoord(),
    FE.ConvenientStoreFeature(conv_store, 500),
    FE.VillageFeature(demographic, li_coord),
    FE.InAlley(),
    FE.BusAggregateFeature(bus, 400),
    FE.TouristSpotFeature(tourist_spot, 400),
    FE.BikeRouteFeaure(bike_route, 1000),
    FE.HighSchoolFeature(highschool, 400),
    FE.MRTIndex(MRT, 200),
    FE.WeatherIndex(weather_df)
])

print("Station Info with Geo Info Added:")
df_geo = fp.build(yb_info)      # ! can't use name as index, or else concat will fail due to index mismatch
fp.show_features()

Station Info with Geo Info Added:
0	XYCoord
1	ConvenientStoreFeature[distance = 500]
2	VillageFeature
3	InAlley
4	BusAggregateFeature[distance = 400]
5	TouristSpotFeature[distance = 400]
6	BikeRouteFeaure[distance = 1000]
7	HighSchoolFeature[distance = 400]
8	MRTIndex[distance = 200]
9	WeatherIndex
Total columns:  27


In [30]:
df_geo

,name,total,lat,lng,address,x,y,conv_count,總計,age_0_15,...,中位數,第一分位數,第三分位數,in_alley,bus_arround,count_tourist_spot,around_riverside,around_highschool,MRT_name,weather_station
0,捷運科技大樓站,28,25.02605,121.54360,復興南路二段235號前,304859.938784,2.768773e+06,9,5289.763742,13.824678,...,613.484660,259.013040,1351.154626,False,57,0,False,True,科技大樓,信義
1,復興南路二段273號前,21,25.02565,121.54357,復興南路二段273號西側,304857.088954,2.768729e+06,11,5298.603467,13.858581,...,613.339145,258.972667,1350.220552,False,55,0,False,True,科技大樓,信義
2,國北教大實小東側門,16,25.02429,121.54124,和平東路二段96巷7號,304622.542775,2.768577e+06,9,5484.350616,14.324470,...,620.931923,261.313565,1364.446159,True,70,0,False,False,None,信義
3,和平公園東側,11,25.02351,121.54282,和平東路二段118巷33號,304782.347456,2.768491e+06,7,5398.918114,14.160731,...,616.023285,259.867428,1352.210162,True,62,1,False,False,None,信義
4,辛亥復興路口西北側,16,25.02153,121.54299,復興南路二段368號,304800.383642,2.768272e+06,5,5323.235098,14.008011,...,610.184094,258.212969,1337.336069,False,26,1,False,False,None,信義
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,臺大獸醫館南側,24,25.01791,121.54242,臺大獸醫系館南側,304744.461695,2.767871e+06,3,5264.655805,13.658091,...,599.359170,255.475791,1309.047082,False,38,2,False,False,None,信義
1307,臺大新體育館東南側,40,25.02112,121.53591,臺大體育館東側,304086.012244,2.768224e+06,2,5339.805774,13.987413,...,610.972571,258.291626,1337.603485,False,26,2,False,False,None,臺北
1308,臺大明達館北側(員工宿舍),18,25.01816,121.54469,明達館北側前空地,304973.457593,2.767900e+06,2,5259.413040,13.589284,...,598.060255,255.019815,1306.648207,False,58,1,False,False,None,信義
1309,辛亥路五段73巷口,23,24.99818,121.55312,已移除。手動增加資訊,305833.314032,2.765690e+06,7,5488.574132,12.967324,...,577.983221,249.638505,1248.837684,False,27,2,False,False,None,文山


In [94]:
yb_flow_add_geoinfo = yb_flow_set_index.join(df_geo.set_index('name')).reset_index()

## With space-time

In [95]:

fp_spacetime = FeaturePipe([
    FE.MRTTimeFeature(MRT),
    FE.WeatherTimeFeature(weather_df),
    FE.FeatureTime(),
    FE.Covid19Feature()
])

fp_spacetime.build(yb_flow_add_geoinfo)
fp_spacetime.df

,name,time,count_out,count_in,neighbor_out,neighbor_in,total,lat,lng,address,...,MRT_inflow,MRT_outflow,temperature,relative_humidity,wind_speed,precipitation,hour,month,weekday,during_covid
0,捷運科技大樓站,2020-04-01 00:00:00,1,1,2,3,28,25.02605,121.54360,復興南路二段235號前,...,4.350112,4.261691,<NA>,<NA>,<NA>,<NA>,0,4,2,False
1,捷運科技大樓站,2020-04-01 01:00:00,0,0,0,1,28,25.02605,121.54360,復興南路二段235號前,...,2.634477,2.716838,20.0,97,1.1,1.0,1,4,2,False
2,捷運科技大樓站,2020-04-01 02:00:00,0,0,0,0,28,25.02605,121.54360,復興南路二段235號前,...,2.245513,2.426511,19.3,96,1.1,0.0,2,4,2,False
3,捷運科技大樓站,2020-04-01 03:00:00,0,0,1,0,28,25.02605,121.54360,復興南路二段235號前,...,2.120574,2.176091,18.7,96,0.3,0.5,3,4,2,False
4,捷運科技大樓站,2020-04-01 04:00:00,0,0,0,0,28,25.02605,121.54360,復興南路二段235號前,...,1.875061,2.096910,18.3,96,0.9,0.5,4,4,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33426562,機場捷運台北車站(6號出口),2023-02-28 19:00:00,4,3,38,34,23,25.04969,121.51201,已移除。手動增加資訊,...,5.484098,5.308946,19.8,76,1.8,0.0,19,2,1,False
33426563,機場捷運台北車站(6號出口),2023-02-28 20:00:00,1,1,33,29,23,25.04969,121.51201,已移除。手動增加資訊,...,5.224807,5.071595,19.1,81,1.6,0.0,20,2,1,False
33426564,機場捷運台北車站(6號出口),2023-02-28 21:00:00,0,2,31,30,23,25.04969,121.51201,已移除。手動增加資訊,...,5.156446,5.025511,18.7,84,1.1,0.0,21,2,1,False
33426565,機場捷運台北車站(6號出口),2023-02-28 22:00:00,1,3,32,21,23,25.04969,121.51201,已移除。手動增加資訊,...,5.003245,4.907954,18.4,85,1.3,0.0,22,2,1,False


In [87]:
fp_spacetime.show_features()

0	MRTTimeFeature
1	WeatherTimeFeature
2	FeatureTime
3	Covid19Feature
Total columns:  39


In [97]:
fp_spacetime.df.to_parquet("feature_added.pq")